In [1]:
# from copy import deepcopy
# from functools import reduce
# from typing import Any, Callable, Dict, List, Tuple, TypeVar
# from nncf import NNCFConfig

# import torch
# import torch.nn as nn
# from utils import supernet,print_number_of_trainable_model_parameters
# from pathlib import Path
# import jstyleson as json
from transformers import (
    DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments,
    AutoTokenizer,PreTrainedTokenizerFast,
    AutoModel,
    BertForSequenceClassification,
    )
# from nncf.common.utils.os import safe_open
from datasets import load_dataset, concatenate_datasets, load_from_disk


/homes/yusx/anaconda3/envs/raffm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch


In [2]:
# from copy import deepcopy
# model_name = "bert-large-uncased-whole-word-masking"
# model = BertForSequenceClassification.from_pretrained(model_name)
# # original_model = deepcopy(model)

In [3]:
# supernet_weights_path = "./metadata/supernet.bin"
# supernet_elasticity_path = (
#     "./metadata/elasticity_state.pt"
# )
# nncf_config_path = "./metadata/nncf_config_1.json"
# nncf_config_path = Path(nncf_config_path).resolve()


In [4]:
# with safe_open(nncf_config_path) as f:
#     loaded_json = json.load(f)
# nncf_config = NNCFConfig.from_dict(loaded_json)
# nncf_config.device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
# train_supernet = supernet.from_checkpoint(
#     model, nncf_config, supernet_elasticity_path, supernet_weights_path
# )

Load architecture configuration from pareto front subnets

In [6]:
# import torch
# subnet_config_file = "metadata/pareto_front_subnets.pth"
# subnet_configs = torch.load(subnet_config_file)
# for config in subnet_configs:
#     for key in config.keys():
#         print("Key: ", key, "\nData: ", config[key])
#     break


In [7]:
# train_supernet.activate_config(subnet_configs[1]["subnet_config"])

In [8]:
# train_supernet.get_macs_for_active_config()

In [9]:
from fedlib.ve import simulator
from fedlib.lib import Server
import fedlib
from utils import DatasetSplitter

## Initialize GLUE Benchmark dataset

In [10]:
dataset_name = 'sst2'
dataset = load_dataset("glue", dataset_name)
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]
test_dataset = dataset["test"]

100%|██████████| 3/3 [00:00<00:00, 429.30it/s]


Split datasets to clients

In [11]:
#split the test set to local datasets for clients
splitter = DatasetSplitter(train_dataset, seed=123)

local_datasets = splitter.split(n=10, replacement=False)

#flush the variable train_dataset
train_dataset = None

In [12]:
print(len(local_datasets[0]))

6734


In [13]:
# data_args = {
# 'n_clients': 100,
# 'partition' : 'noniid-labeldir',
# 'dataset' : 'cifar10',
# 'datadir' : './data',
# 'beta' : .5,
# 'train_bs' : 64,
# 'test_bs' : 128,
# 'n_worker' : 32
# }

# data_loaders, global_test_dl, _ = fedlib.init_dataset(**data_args)


# Define the Model to Optimize in FL
Here I use the `bert-large-uncased-whole-word-masking` as an example.

In [14]:
model_name = 'bert-large-uncased-whole-word-masking'
from transformers import AutoConfig

# The identifier name of the pre-trained model
model_name = 'bert-large-uncased-whole-word-masking'

# Load the configuration from the identifier
config = AutoConfig.from_pretrained(model_name)



# The config object now contains the configuration of the BERT model
config.num_labels = 2

# print(config)


In [15]:
import copy
model = BertForSequenceClassification.from_pretrained(model_name, config=config)

# print(model)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-un

In [16]:
tokenizer = AutoTokenizer.from_pretrained(
   model_name,
   use_fast=True,
)
# print(tokenizer)

In [17]:
from utils import tokenize_function

In [18]:
# tokenized_train_dataset = train_dataset.map(lambda examples: tokenize_function(examples, tokenizer, dataset_name), batched=True)
tokenized_test_dataset = test_dataset.map(lambda examples: tokenize_function(examples, tokenizer, dataset_name), batched=True)
# logging.info("=====> train_dataset size: {}".format(len(tokenized_train_dataset)))
print("=====> tokenized_test_dataset size: {}".format(len(tokenized_test_dataset)))

=====> tokenized_test_dataset size: 1821


In [19]:
communication_round = 100
for i in range(0,communication_round):
    for client_id in range(0,len(local_datasets)):
        # tokenized_local_train_dataset = local_datasets[client_id].map(lambda examples: tokenize_function(examples, tokenizer, dataset_name), batched=True)
        tokenized_local_train_dataset = [tokenize_function(example,tokenizer, dataset_name) for example in local_datasets[client_id]]
        
        print("=====> tokenized_local_train_dataset size: {}".format(len(tokenized_local_train_dataset)))
        break
    break

=====> tokenized_local_train_dataset size: 6734


In [ ]:
training_args = TrainingArguments(
    output_dir='./log_dir',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./log_dir',
    save_total_limit = 1,
    save_strategy = "no",
    evaluation_strategy="epoch",
    )

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`

In [ ]:

from sklearn.metrics import accuracy_score
from scipy.stats import pearsonr

In [ ]:
def compute_metrics(eval_pred, task):
    predictions, labels = eval_pred
    if task == "stsb":
        pearson_corr, _ = pearsonr(predictions.squeeze(), labels)
        return {"pearson_corr": pearson_corr}
    else:
    
        predictions = predictions.argmax(-1)
        return {"accuracy": accuracy_score(labels, predictions)}